In [1]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np
from random import random, randint, seed, choice


In [2]:

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'minDistToFood', 'distToPill', 'minDistEdibleGhost']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: |ghostX - pacmanX| + |ghostY - pacmanY| "
" minDistToFood: |foodX - pacmanX| + |foodY - pacmanY| "
" distToPill: |pillX - pacmanX| + |pillY - pacmanY| "


       

' distToPill: |pillX - pacmanX| + |pillY - pacmanY| '

In [3]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo):
    return trainIndividual(individuo)

def getPopFitness(population):
    return [getFitness(population[i]) for i in range(0,len(population))]

    
    




In [44]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def minDistBFS(agentPos, positions, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if len(positions) > 0:
        while queue:
            el = queue.pop(0)
            h = el[1]
            s = el[0]

            north = (s[0],s[1]+1)
            south = (s[0],s[1]-1)
            west = (s[0]-1,s[1])
            east = (s[0]+1,s[1])

            for i in [north, south, west, east]:    
                if visited[i[0]][i[1]] == False:
                    queue.append((i,h+1))
                    visited[i[0]][i[1]] = True

                    if(i in positions):    
    #                     print (i, end = " ")
                        return h
                
    return 0

def findFoodMinDistBFS(agentPos, foodGrid, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True

    while queue:
        el = queue.pop(0)
        h = el[1]
        s = el[0]

        north = (s[0],s[1]+1)
        south = (s[0],s[1]-1)
        west = (s[0]-1,s[1])
        east = (s[0]+1,s[1])

        for i in [north, south, west, east]:    
            if visited[i[0]][i[1]] == False:
                queue.append((i,h+1))
                visited[i[0]][i[1]] = True

                if(foodGrid[i[0]][i[1]]):    
#                     print (i, end = " ")
                    return h
                
    return 0

def getMinDistEdibleGhost(agentPos,state):
    ghostPos = []
    ghostStates = state.getGhostStates()
    for gs in ghostStates:
        if gs.scaredTimer > 0:
            ghostPos.append(gs.getPosition())
            
    return minDistBFS(agentPos, ghostPos, state.getWalls())

# def calcFeatures(state, action):
#     futureState = state.generatePacmanSuccessor(action)
#     print("@@ ",state.getLegalActions(),"   -  ",futureState.getLegalActions())
            
#     futureScore = futureState.getScore() - state.getScore()
#     eatFood = state.getNumFood() - futureState.getNumFood()
#     ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
#     foodDist = findFoodMinDistBFS(futureState.getPacmanPosition(),futureState.getFood(),state.getWalls())
#     pillDist = calcMinPillDist(futureState.getPacmanPosition(),futureState.getCapsules())
# #             stateVars[0] = futureScore
# #             stateVars[1] = eatFood
# #             stateVars[2] = ghostDist
# #             stateVars[3] = foodDist
# #             stateVars[4] = pillDist
#     print("STATE VARS: FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist)
#     return np.array([[futureScore/10,eatFood,ghostDist/10,foodDist/10,pillDist/10]])
# directionsAux = {}
# directionsAux['North'] = (0,+1)
# directionsAux['South'] = (0,-1)
# directionsAux['West'] = (-1,0)
# directionsAux['East'] = (+1,0)
def calcFeatures(state, action):
    futureState = state.generatePacmanSuccessor(action)
#     print("@@ ",state.getLegalActions(),"   -  ")
#     d = directionsAux[action]
    pos = state.getPacmanPosition()
    
#     future_pos = (pos[0]+d[0],pos[1]+d[1])
    future_pos = futureState.getPacmanPosition()
    
    if state.getFood()[future_pos[0]][future_pos[1]] or (future_pos in state.getCapsules()):
        eatFood = 1.0
    else:
        eatFood = 0
        
    
    futureScore = futureState.getScore() - state.getScore()
#     eatFood = 1.0 if  else 0.0
    ghostDist = minDistBFS(future_pos, futureState.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(future_pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(future_pos, state.getCapsules(), state.getWalls())
    edibleGhost = getMinDistEdibleGhost(future_pos,futureState)

    print("STATE VARS:  FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist," EG: ",edibleGhost)
    return np.array([[futureScore/250,eatFood,ghostDist/30,foodDist/30,pillDist/30,edibleGhost/20]])

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, W = None, Q=0):
        self.W = W
        self.Q = Q
        
    def setW(self, W):
        self.W = W
        
    def setQ(self, Q):
        self.Q = Q

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
#         print(state.getLegalPacmanActions())
#         print(state.getPacmanState())

        print("PACMANPOS0 :", state.getPacmanPosition())
#         print(state.getNumFood())
#         print(state.getScore())
        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        scoresAux = []
        
        for action in legalActions:    
            stateVars = calcFeatures(state, action)
            actionValue = np.dot(self.W,stateVars.T)
            actionScore.append(actionValue[0][0])
            if actionValue[0][0] > 0:
                scoresAux.append(actionValue[0][0])
            else:
                scoresAux.append(0.0001)
        print("Action Score: ",actionScore)
        print(state.getLegalPacmanActions())
#         if actionScore.count(max(actionScore)) > 1:
#             maxScore = max(actionScore)
#             indexes = [i for i, j in enumerate(actionScore) if j == maxScore]
#             maxIndex = choice(indexes)
#         else:
        scoresAux = actionScore[:]
        if min(scoresAux) < 0:
            scoresAux = scoresAux[:]-min(scoresAux)
            if len(scoresAux) < 2 or sum(scoresAux[:]) == 0:
                scoresAux = scoresAux[:]+1
        
        
        norm = np.array(scoresAux[:]/sum(scoresAux[:]))
        print("scoresAux: ",scoresAux, "  -  norm: ",norm)
        action = np.random.choice(legalActions, p=norm)
            
#         action = legalActions[maxIndex]
        
        gama = 0.9
        r = 0
        alfa = 0.1
        
        difference = (r+gama*max(actionScore)) - self.Q
        self.setQ(max(actionScore))
#         print(state.getLegalPacmanActions()," ACTION: ",action)
        self.setW(self.W + alfa*difference*calcFeatures(state, action))
        
        print("W: ", self.W, "  -  ACTION: ",action)
#         print("Action: ",action)
#         futureState = state.generatePacmanSuccessor(action)
#         futureScore = futureState.getScore() - state.getScore()
#         eatFood = state.getNumFood() - futureState.getNumFood()
#         ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
#         foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
#         print("STATE VARS: FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist)
        
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [46]:
layoutType='mediumClassic'
numGames=20
gameLayout = layout.getLayout( layoutType )
print("------------------------------------------ train --------------------")

W = np.zeros((1,6))
W[:] = 0.7
pacman = MyGoWestAgent(W)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

# import textDisplay
# textDisplay.SLEEP_TIME = 0.1
# gameDisplay = textDisplay.PacmanGraphics()
import graphicsDisplay
gameDisplay = graphicsDisplay.PacmanGraphics(1.0, frameTime = 0.1)

games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
scores = [game.state.getScore() for game in games]
wins = [game.state.isWin() for game in games]
winRate = wins.count(True)/ float(len(wins))
avgScore = sum(scores) / float(len(scores))

print(pacman.W)
print(avgScore)

layout:  mediumClassic
------------------------------------------ train --------------------
PACMANPOS0 : (9, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  15  FD:  1  PD:  12  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  14  FD:  1  PD:  14  EG:  0
Action Score:  [1.3785333333333332, 1.4018666666666668]
['East', 'West', 'Stop']
scoresAux:  [1.3785333333333332, 1.4018666666666668]   -  norm:  [0.49580396 0.50419604]
STATE VARS:  FS:  9.0  EF:  1.0  GD:  15  FD:  1  PD:  12  EG:  0
W:  [[0.70454205 0.826168   0.763084   0.7042056  0.7504672  0.7       ]]   -  ACTION:  East
PACMANPOS0 : (10, 1)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  11  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  14  FD:  1  PD:  13  EG:  0
Action Score:  [1.480846073728, 0.701963671808]
['East', 'West', 'Stop']
scoresAux:  [1.480846073728, 0.701963671808]   -  norm:  [0.6784128 0.3215872]
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  11  EG:  0
W:  [[0.70429327 0.81925748 0.76008944 0.70397525 0.7479

PACMANPOS0 : (11, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  13  FD:  2  PD:  10  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  3  PD:  12  EG:  0
Action Score:  [0.5629263661061469, 0.5851670549959456]
['East', 'West', 'Stop']
scoresAux:  [0.5629263661061469, 0.5851670549959456]   -  norm:  [0.49031408 0.50968592]
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  3  PD:  12  EG:  0
W:  [[0.7061291  0.84164637 0.67904924 0.68372163 0.67008716 0.7       ]]   -  ACTION:  West
PACMANPOS0 : (10, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  3  PD:  11  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  9  FD:  2  PD:  13  EG:  0
Action Score:  [0.5602309947528955, 0.5368428021398741]
['East', 'West', 'Stop']
scoresAux:  [0.5602309947528955, 0.5368428021398741]   -  norm:  [0.51065935 0.48934065]
STATE VARS:  FS:  -1.0  EF:  0  GD:  9  FD:  2  PD:  13  EG:  0
W:  [[0.70616149 0.84164637 0.67662046 0.6831819  0.66657893 0.7       ]]   -  ACTION:  West
PACMANPOS0 : (9, 1)
STATE VARS:  FS:  -1.0  

PACMANPOS0 : (6, 6)
STATE VARS:  FS:  -1.0  EF:  0  GD:  3  FD:  1  PD:  7  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  1  FD:  1  PD:  9  EG:  0
Action Score:  [0.2336711397008357, 0.23275065979226486]
['North', 'South', 'Stop']
scoresAux:  [0.2336711397008357, 0.23275065979226486]   -  norm:  [0.50098675 0.49901325]
STATE VARS:  FS:  -1.0  EF:  0  GD:  3  FD:  1  PD:  7  EG:  0
W:  [[0.70785122 0.86314597 0.64149096 0.67538484 0.61460179 0.7       ]]   -  ACTION:  North
PACMANPOS0 : (6, 7)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  8  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  1  FD:  2  PD:  8  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  3  FD:  1  PD:  8  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  2  FD:  1  PD:  6  EG:  0
Action Score:  [1.1391843470496077, 0.22747109463788362, 0.24772433024408824, 1.076827862335127]
['North', 'South', 'East', 'West', 'Stop']
scoresAux:  [1.1391843470496077, 0.22747109463788362, 0.24772433024408824, 1.076827862335127]   -  norm:  [0.

PACMANPOS0 : (9, 9)
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  2  PD:  13  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  4  PD:  11  EG:  0
Action Score:  [0.5052390246315406, 0.47519221541909074]
['East', 'West', 'Stop']
scoresAux:  [0.5052390246315406, 0.47519221541909074]   -  norm:  [0.51532326 0.48467674]
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  2  PD:  13  EG:  0
W:  [[0.71713633 1.07817018 0.57537105 0.66362153 0.53238067 0.7       ]]   -  ACTION:  East
PACMANPOS0 : (10, 9)
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  1  PD:  14  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  3  PD:  12  EG:  0
Action Score:  [0.4978449064158057, 0.468236226987932]
['East', 'West', 'Stop']
scoresAux:  [0.4978449064158057, 0.468236226987932]   -  norm:  [0.51532412 0.48467588]
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  1  PD:  14  EG:  0
W:  [[0.7171592  1.07817018 0.57308391 0.66343094 0.52971234 0.7       ]]   -  ACTION:  East
PACMANPOS0 : (11, 9)
STATE VARS:  FS:  9.0 

PACMANPOS0 : (13, 9)
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  12  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  14  EG:  0
Action Score:  [1.5556619138522556, 0.3797648673231131]
['South', 'West', 'Stop']
scoresAux:  [1.5556619138522556, 0.3797648673231131]   -  norm:  [0.80378236 0.19621764]
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  12  EG:  0
W:  [[0.72475002 1.2646636  0.5522922  0.65318644 0.50582211 0.7       ]]   -  ACTION:  South
PACMANPOS0 : (13, 8)
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  13  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  11  EG:  0
Action Score:  [0.38870452503782266, 1.6636832488300963]
['North', 'South', 'Stop']
scoresAux:  [0.38870452503782266, 1.6636832488300963]   -  norm:  [0.18939137 0.81060863]
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  11  EG:  0
W:  [[0.72453997 1.2588289  0.55054179 0.65299195 0.50368272 0.7       ]]   -  ACTION:  South
PACMANPOS0 : (13, 7)
STATE VARS:  FS:

...

PACMANPOS0 : (9, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  14  FD:  1  PD:  12  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  15  FD:  1  PD:  14  EG:  0
Action Score:  [1.3291937001054286, 0.43092794099118814]
['East', 'West', 'Stop']
scoresAux:  [1.3291937001054286, 0.43092794099118814]   -  norm:  [0.7551715 0.2448285]
STATE VARS:  FS:  9.0  EF:  1.0  GD:  15  FD:  1  PD:  14  EG:  0
W:  [[-1.90680931 -0.77518299  6.13539557 -1.00061034 -3.770719   79.82587505]]   -  ACTION:  West
PACMANPOS0 : (8, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  13  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  14  EG:  0
Action Score:  [1.3746394320982136, 0.021820747607672608]
['East', 'West', 'Stop']
scoresAux:  [1.3746394320982136, 0.021820747607672608]   -  norm:  [0.98437424 0.01562576]
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  13  EG:  0
W:  [[-1.90677251 -0.77518299  6.13079466 -1.0012238  -3.77470646 79.82587505]]   -  ACTION:  East
PACMANPOS0 : (9, 1)
STATE

PACMANPOS0 : (9, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  1  PD:  12  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  1  PD:  14  EG:  0
Action Score:  [0.44844063323667127, 0.5948841005942547]
['East', 'West', 'Stop']
scoresAux:  [0.44844063323667127, 0.5948841005942547]   -  norm:  [0.42981885 0.57018115]
STATE VARS:  FS:  -1.0  EF:  0  GD:  12  FD:  1  PD:  14  EG:  0
W:  [[-1.90607294 -0.77518299  6.06903782 -1.01655954 -3.85055328 79.82587505]]   -  ACTION:  West
PACMANPOS0 : (8, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  2  PD:  13  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  14  EG:  0
Action Score:  [0.29429317514150277, -0.4492979251157738]
['East', 'West', 'Stop']
scoresAux:  [0.7435911 0.       ]   -  norm:  [1. 0.]
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  2  PD:  13  EG:  0
W:  [[-1.90594094 -0.77518299  6.05803715 -1.01875967 -3.86485416 79.82587505]]   -  ACTION:  East
PACMANPOS0 : (9, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  8  FD

PACMANPOS0 : (8, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  13  EG:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  14  EG:  0
Action Score:  [1.3060311169848033, -0.044799136656229255]
['East', 'West', 'Stop']
scoresAux:  [1.35083025 0.        ]   -  norm:  [1. 0.]
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  13  EG:  0
W:  [[-1.90604649 -0.77518299  6.06949111 -1.02137066 -3.85887628 79.82587505]]   -  ACTION:  East
PACMANPOS0 : (9, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  1  PD:  12  EG:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  13  FD:  1  PD:  14  EG:  0
Action Score:  [1.464773541134752, 0.8028823815186368]
['East', 'West', 'Stop']
scoresAux:  [1.464773541134752, 0.8028823815186368]   -  norm:  [0.64594171 0.35405829]
STATE VARS:  FS:  -1.0  EF:  0  GD:  13  FD:  1  PD:  14  EG:  0
W:  [[-1.9060514  -0.77518299  6.0700226  -1.02132977 -3.85830391 79.82587505]]   -  ACTION:  West
PACMANPOS0 : (8, 1)
STATE VARS:  FS:  -1.0  EF:  0  GD:  13  FD

SystemExit: 0

/home/miguel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
# GAME LAYOUTS: smallClassic, mediumClassic, originalClassic
gameLayout = layout.getLayout( 'mediumClassic' )

ind = GPTree()
ind.random_tree(True, 2)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)

layout:  mediumClassic


NameError: name 'GPTree' is not defined